In [ ]:
 

Hi Sean,

 

As requested, these are the tables from Mix; namely

Asset Master  (Daily Stamp)
Driver Master  (Daily Stamp)
Events Library
Events  (extracted every hour and stored. The last reloads run just before midnight which extracts all the days data)
Position    (stored every 30 seconds)
Trips (extracted every hour and stored. The last reloads run just before midnight which extracts all the days data)
 
Fields that are highlighted in yellow in the attached files, are fields that are brought in for easy reference and these are the Organisation Name, Driver Name, Asset Registration and Vin Number and Events description.

 

Please note that for Mix, we have data only available for the current day and stored as highlighted in the red font above

 

 

 

In [1]:
import xlrd

In [31]:
import sys
import re
import pandas as pd
import numpy as np

In [7]:
data_path='/media/lnr-ai/christo/github_repos/logistics/data/'

In [26]:
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

In [9]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]
onlyfiles

['Asset Master.xlsx',
 'Driver Master.xlsx',
 'Events Library Master.xlsx',
 'Events.xlsx',
 'Position.xlsx',
 'Trips.xlsx']

In [11]:
trips_df=pd.read_excel('/media/lnr-ai/christo/github_repos/logistics/data/Trips.xlsx', sheet_name='Sheet1')

In [13]:
list(trips_df)

['GroupId',
 'Organisation Name',
 'Source',
 'DriverId',
 'Driver Name',
 'AssetId',
 'VinNumber',
 'RegistrationNumber',
 '__KEY_root',
 'DistanceKilometers',
 'DrivingTime',
 'Duration',
 'EndOdometerKilometers',
 'EndPositionId',
 'FirstDepart',
 'FuelUsedLitres',
 'LastHalt',
 'MaxAccelerationKilometersPerHourPerSecond',
 'MaxDecelerationKilometersPerHourPerSecond',
 'MaxRpm',
 'MaxSpeedKilometersPerHour',
 'PulseValue',
 'StandingTime',
 'StartOdometerKilometers',
 'StartPositionId',
 'TripEnd',
 'TripId',
 'TripStart']

In [16]:
for filename in onlyfiles:
    print(filename)
    df=pd.read_excel('/media/lnr-ai/christo/github_repos/logistics/data/{filename}'.format(filename=filename), sheet_name='Sheet1')
    print(list(df))

Asset Master.xlsx
['GroupId', 'Organisation Name', 'Source', 'AdditionalMobileDevice', 'AssetId', 'AssetImage', 'AssetImageUrl', 'AssetTypeId', 'Colour', 'CreatedBy', 'CreatedDate', 'CreatedTime', 'Description', 'EngineHours', 'EngineNumber', 'FleetNumber', 'FmVehicleId', 'FuelType', 'Icon', 'IconColour', 'IsConnectedTrailer', 'IsDefaultImage', 'Make', 'Model', 'Notes', 'Odometer', 'RegistrationNumber', 'TargetFuelConsumption', 'TargetFuelConsumptionUnits', 'TargetHourlyFuelConsumption', 'TargetHourlyFuelConsumptionUnits', 'UserState', 'VinNumber', 'Year']
Driver Master.xlsx
['GroupId', 'Organisation Name', 'Source', 'Driver Name', 'DriverId', 'EmployeeNumber', 'ExtendedDriverIdType', 'FmDriverId', 'IsSystemDriver', 'MobileNumber']
Events Library Master.xlsx
['GroupId', 'Organisation Name', 'Source', 'Event Description', 'EventType', 'EventTypeId']
Events.xlsx
['Name', 'Description', 'EventType', '__KEY_root', 'Armed', 'AssetId_u0', 'DriverId_u0', 'EventDateTime', 'EventId', 'EventLati

### Asset Master.xlsx

Asset Master.xlsx
['GroupId', 'Organisation Name', 'Source', 'AdditionalMobileDevice', 'AssetId', 'AssetImage', 'AssetImageUrl', 'AssetTypeId', 'Colour', 'CreatedBy', 'CreatedDate', 'CreatedTime', 'Description', 'EngineHours', 'EngineNumber', 'FleetNumber', 'FmVehicleId', 'FuelType', 'Icon', 'IconColour', 'IsConnectedTrailer', 'IsDefaultImage', 'Make', 'Model', 'Notes', 'Odometer', 'RegistrationNumber', 'TargetFuelConsumption', 'TargetFuelConsumptionUnits', 'TargetHourlyFuelConsumption', 'TargetHourlyFuelConsumptionUnits', 'UserState', 'VinNumber', 'Year']

### Driver Master.xlsx

['GroupId', 'Organisation Name', 'Source', 'Driver Name', 'DriverId', 'EmployeeNumber', 'ExtendedDriverIdType', 'FmDriverId', 'IsSystemDriver', 'MobileNumber']

### Events Library Master.xlsx

['Name', 'Description', 'EventType', '__KEY_root', 'Armed', 'AssetId_u0', 'DriverId_u0', 'EventDateTime', 'EventId', 'EventLatitude', 'EventLongitude', 'EventSpeed', 'EventTotalTime', 'EventTypeId', 'GroupId', 'OdometerKilometres_u0', 'OdometerKilometresEnd_u0', 'Priority', 'Source', 'Value']

### Position.xlsx

['GroupId', 'Organisation Name', 'Source', 'DriverId', 'Driver Name', 'AssetId', 'VinNumber', 'RegistrationNumber', 'ageOfReadingSeconds', 'altitudeMetres', 'distanceSinceReadingKilometres', 'EventDate', 'EventDateTime', 'EventHour', 'Eventtime', 'formattedAddress', 'hdop', 'heading', 'isAvl', 'latitude', 'longitude', 'numberOfSatellites', 'odometerKilometres', 'pdop', 'positionId', 'speedKilometresPerHour', 'speedLimit', 'timestamp', 'vdop']

### Trips.xlsx

['GroupId', 'Organisation Name', 'Source', 'DriverId', 'Driver Name', 'AssetId', 'VinNumber', 'RegistrationNumber', '__KEY_root', 'DistanceKilometers', 'DrivingTime', 'Duration', 'EndOdometerKilometers', 'EndPositionId', 'FirstDepart', 'FuelUsedLitres', 'LastHalt', 'MaxAccelerationKilometersPerHourPerSecond', 'MaxDecelerationKilometersPerHourPerSecond', 'MaxRpm', 'MaxSpeedKilometersPerHour', 'PulseValue', 'StandingTime', 'StartOdometerKilometers', 'StartPositionId', 'TripEnd', 'TripId', 'TripStart']

In [23]:
filename='Driver Master.xlsx'
driver_master_df=pd.read_excel('/media/lnr-ai/christo/github_repos/logistics/data/{filename}'.format(filename=filename), sheet_name='Sheet1')


In [25]:
set(driver_master_df.DriverId)

{nan,
 3.3305698794800824e+18,
 6.652289605037818e+18,
 8.093161458326888e+18,
 7.169467641638535e+18,
 8.744478296203829e+18,
 8.726659782798377e+18,
 nan,
 3.8059750176701153e+18,
 8.132445079506649e+18,
 8.952341289603039e+18,
 4.910083541040103e+18,
 7.405450040768668e+18,
 3.9915576974804255e+18,
 3.626017972311818e+18,
 -6.626867539558152e+17,
 6.061962310230475e+18,
 6.219844380796767e+17,
 -8.050844404945157e+18,
 -3.3719859843360676e+18,
 -4.0228169960390477e+18,
 4.7743004033295e+18,
 -1.7260087770960184e+18,
 4.266749141649475e+18,
 -4.810468960488716e+18,
 -7.442358142579735e+17,
 -3.302124514094938e+18,
 -4.6562565332600416e+17,
 -2.5164527313362938e+17,
 -4.003497403919564e+17,
 4.2713093013209764e+18,
 3.3673241668030976e+18,
 6.026843393298729e+18,
 6.396936621783976e+18,
 4.702256903919962e+18,
 1.615159226755304e+18,
 -5.903466581901932e+18,
 -6.017375675564652e+18,
 -2.5272267380059965e+18,
 -3.9492579078585354e+18,
 3.1723743285480356e+18,
 -2.7605958128016225e+18,


In [18]:
filename='Events Library Master.xlsx'
events_library_master_df=pd.read_excel('/media/lnr-ai/christo/github_repos/logistics/data/{filename}'.format(filename=filename), sheet_name='Sheet1')


In [20]:
list(events_library_master_df)

['GroupId',
 'Organisation Name',
 'Source',
 'Event Description',
 'EventType',
 'EventTypeId']

In [21]:
filename='Events.xlsx'
events_df=pd.read_excel('/media/lnr-ai/christo/github_repos/logistics/data/{filename}'.format(filename=filename), sheet_name='Sheet1')


In [22]:
set(events_df.DriverId_u0)

{nan,
 nan,
 6.366881122800517e+18,
 9.108944395108016e+18,
 6.215628227223372e+18,
 8.323304274917835e+18,
 9.77250791234216e+17,
 8.998042535176602e+18,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 3.1723743285480356e+18,
 -4.0381738421844664e+18,
 1.0256276274522605e+18,
 1.0124015246410712e+18,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 1.3071701446926502e+16,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 4.5632222382230477e+18,
 1.9217582021590223e+18,
 6.225612698741885e+17,
 -8.541184624809981e+17,
 1.26734095467638e+18,
 -6.824364709692627e+18,
 -4.281541024229415e+18,
 -8.629775268147745e+18,
 8.885926099329765e+17,
 1.0283813638631711e+18,
 9.660091279767101e+17,
 6.402685439517012e+18,
 -2.794218548286992e+17,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 2.2195715149128547e+17,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 -7.614076142884733e+18,
 1

In [ ]:
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

In [28]:
driverset=list(set(intersection(list(events_df.DriverId_u0),list(driver_master_df.DriverId))))

In [29]:
len(driverset)

682

In [33]:
events_df.EventId

0         2.804730e+18
1         2.804730e+18
2         2.804427e+18
3         2.804427e+18
4         2.804434e+18
              ...     
114827             NaN
114828             NaN
114829             NaN
114830             NaN
114831             NaN
Name: EventId, Length: 114832, dtype: float64

In [143]:
set(events_df.E)

{'Custom',
 'Default',
 'Diagnostic',
 'Hidden',
 'HosViolation',
 'ServerSide',
 'System'}

In [36]:
len(set(events_df.EventTypeId))

436

### Group driver by event

In [151]:
events_df.Description

0                   .Harsh Braking > 8km/h/s (MV)(Trucks)
1                   .Harsh Braking > 8km/h/s (MV)(Trucks)
2                   .Harsh Braking > 8km/h/s (MV)(Trucks)
3                   .Harsh Braking > 8km/h/s (MV)(Trucks)
4                   .Harsh Braking > 8km/h/s (MV)(Trucks)
                               ...                       
114827        xxxMobileye CAN-A Warning Forward Collision
114828      xxxMobileye CAN-A Warning Left Lane Departure
114829     xxxMobileye CAN-A Warning Pedestrian Collision
114830    xxxMobileye CAN-A Warning Pedestrian Collisionx
114831                 xxxTazz Prohibited Driving (Sum) x
Name: Description, Length: 114832, dtype: object

In [152]:
events_df_grouped = events_df.groupby(['DriverId_u0', 'Description'])

In [153]:
events_df_grouped_count=events_df_grouped.aggregate(np.size)

In [154]:
events_df_grouped_count.reset_index(inplace=True)

In [155]:
events_df_grouped_count

,DriverId_u0,Description,Name,EventType,__KEY_root,Armed,AssetId_u0,EventDateTime,EventId,EventLatitude,EventLongitude,EventSpeed,EventTotalTime,EventTypeId,GroupId,OdometerKilometres_u0,OdometerKilometresEnd_u0,Priority,Source,Value
0,-9.174768e+18,*After Hour,4,4,4.0,4.0,4.0,4,4.0,4.0,4.0,4.0,4.0,4,4,4.0,4.0,4.0,4,4.0
1,-9.174768e+18,.TAR - Excessive Idling > 10min,4,4,4.0,4.0,4.0,4,4.0,4.0,4.0,4.0,4.0,4,4,4.0,4.0,4.0,4,4.0
2,-9.174768e+18,GSM Status - Possible Jamming,100,100,100.0,100.0,100.0,100,100.0,100.0,100.0,100.0,100.0,100,100,100.0,100.0,100.0,100,100.0
3,-9.174768e+18,Idle,204,204,204.0,204.0,204.0,204,204.0,204.0,204.0,204.0,204.0,204,204,204.0,204.0,204.0,204,204.0
4,-9.174768e+18,Over speeding,126,126,126.0,126.0,126.0,126,126.0,126.0,126.0,126.0,126.0,126,126,126.0,126.0,126.0,126,126.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1833,9.218301e+18,Over Speeding (Reports),4,4,4.0,4.0,4.0,4,4.0,4.0,4.0,4.0,4.0,4,4,4.0,4.0,4.0,4,4.0
1834,9.218301e+18,Road Speed Overspeeding,20,20,20.0,20.0,20.0,20,20.0,20.0,20.0,20.0,20.0,20,20,20.0,20.0,20.0,20,20.0
1835,9.220211e+18,Idle - excessive,2,2,2.0,2.0,2.0,2,2.0,2.0,2.0,2.0,2.0,2,2,2.0,2.0,2.0,2,2.0
1836,9.220211e+18,Over revving,36,36,36.0,36.0,36.0,36,36.0,36.0,36.0,36.0,36.0,36,36,36.0,36.0,36.0,36,36.0


In [156]:
events_df_grouped_pivot=events_df_grouped_count.pivot_table(index='DriverId_u0', columns='Description', values='GroupId',fill_value=0)

In [157]:
events_df_grouped_pivot

Description,*After Hour,*Asset in Workshop Hours,.Harsh Braking > 8km/h/s (MV)(Trucks),.Harsh Braking >10km/h/s (LDV/Fuso F) (MV),.Impact Detected >13 (MV),.TAR - Alert Possible Accident >20km/h/s,.TAR - Excessive Idling > 10min,.TAR - Over Speeding > 100km/h (Trucks),Battery disconnection,GPS Jamming Detected - Critical,...,Harsh braking,Harsh braking (Reports),Idle,Idle - excessive,Ignition On [Activate Relay Drive Off),Over Speeding (Reports),Over revving,Over revving (Reports),Over speeding,Road Speed Overspeeding
DriverId_u0,,,,,,,,,,,,,,,,,,,,,
-9.174768e+18,4,0,0,0,0,0,4,0,0,0,...,0,0,204,0,0,0,0,0,126,156
-9.145267e+18,0,0,4,0,0,0,0,0,0,0,...,0,2,0,0,0,82,0,0,0,18
-9.124200e+18,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,10,0,0,0,90
-9.107084e+18,0,0,8,0,0,0,0,0,0,0,...,0,2,0,0,0,6,0,0,0,8
-9.101669e+18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,66,0,0,0,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9.132976e+18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,12,0,0,0,14
9.147461e+18,0,0,4,0,0,0,0,0,0,0,...,0,0,0,0,0,4,0,0,0,170
9.153934e+18,0,0,4,0,0,0,0,0,0,0,...,0,0,0,0,0,82,0,0,0,34


In [147]:
EventTypeId_list=list(set(events_df.EventTypeId))

In [149]:
events_library_master_df[events_library_master_df.EventTypeId==EventTypeId_list[0]]

,GroupId,Organisation Name,Source,Event Description,EventType,EventTypeId
76,-7124170799261977621,Bakers Transport (R),ENT,.TAR - No Stopping Across the Road @ W/pool Is...,ServerSide,1017488558022090752
530,-7341754787231762831,Pindulo VDM (Pty) Ltd,ENT,.TAR - No Stopping Across the Road @ W/pool Is...,ServerSide,1017488558022090752


In [119]:
gkk=events_df[['DriverId_u0', 'EventTypeId']].groupby(by=['DriverId_u0','EventTypeId']).count()

In [120]:
gkk.reset_index(inplace=True)

In [121]:
gkk

,DriverId_u0,EventTypeId
0,-9.174768e+18,-3890646499157906515
1,-9.174768e+18,-3393530750645328945
2,-9.174768e+18,-2864371101042705250
3,-9.174768e+18,-2635586025599679100
4,-9.174768e+18,-1345267921410597852
...,...,...
1833,9.218301e+18,-3890646499157906515
1834,9.218301e+18,-2864371101042705250
1835,9.220211e+18,-7372181092478897411
1836,9.220211e+18,-3890646499157906515


In [110]:
gkk[gkk.DriverId_u0==gkk.iloc[0]['DriverId_u0']]

,DriverId_u0,EventTypeId
0,-9.174768e+18,594


In [102]:
gf=gkk.pivot_table(index='DriverId_u0', columns='EventTypeId', 
                        aggfunc=len, fill_value=0)

In [103]:
gf.reset_index(inplace=True)

In [104]:
gf.shape

(682, 218)

In [101]:
gf

,DriverId_u0
EventTypeId,
0,-9.174768e+18
1,-9.145267e+18
2,-9.124200e+18
3,-9.107084e+18
4,-9.101669e+18
...,...
677,9.132976e+18
678,9.147461e+18
679,9.153934e+18


In [93]:
len(set(gf.DriverId_u0))

682

In [96]:
gf.iloc[0].max()

1.0

In [75]:
gkk.iloc[0].DriverId_u0

-9.174768423532061e+18

In [77]:
events_df[events_df.DriverId_u0==gkk.iloc[0].DriverId_u0]

,Name,Description,EventType,__KEY_root,Armed,AssetId_u0,DriverId_u0,EventDateTime,EventId,EventLatitude,EventLongitude,EventSpeed,EventTotalTime,EventTypeId,GroupId,OdometerKilometres_u0,OdometerKilometresEnd_u0,Priority,Source,Value
930,Bakers Transport (R),.TAR - Excessive Idling > 10min,Custom,NaN,NaN,8.914219e+17,-9.174768e+18,2020-09-10T01:17:10,2.804651e+18,-27.113083,22.852167,0.0,0.0,-1345267921410597852,-7124170799261977621,231192.7,NaN,NaN,ENT,0.0
931,Bakers Transport (R),.TAR - Excessive Idling > 10min,Custom,NaN,NaN,8.914219e+17,-9.174768e+18,2020-09-10T01:17:11,2.804651e+18,-27.113060,22.852171,0.0,0.0,-1345267921410597852,-7124170799261977621,231192.7,NaN,NaN,ENT,NaN
1059,Bakers Transport (R),*After Hour,ServerSide,NaN,NaN,8.914219e+17,-9.174768e+18,2020-09-09T17:46:44,2.804535e+18,-27.591305,22.966555,7.0,0.0,969341969527513088,-7124170799261977621,231125.8,NaN,NaN,ENT,7.0
1060,Bakers Transport (R),*After Hour,ServerSide,NaN,NaN,8.914219e+17,-9.174768e+18,2020-09-10T02:55:08,2.804676e+18,-27.113194,22.851973,13.0,0.0,969341969527513088,-7124170799261977621,0.0,NaN,NaN,ENT,13.0
2029,Bakers Transport (R),GSM Status - Possible Jamming,Custom,NaN,NaN,8.914219e+17,-9.174768e+18,2020-09-09T05:44:25,2.804348e+18,-31.686750,18.697420,76.0,18.0,-2635586025599679100,-7124170799261977621,230355.3,NaN,NaN,ENT,255.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106590,Pindulo VDM (Pty) Ltd,Road Speed Overspeeding,Default,NaN,NaN,8.914219e+17,-9.174768e+18,2020-09-10T10:16:40,2.804790e+18,-28.233749,23.560223,67.0,0.0,-2864371101042705250,-7341754787231762831,0.0,NaN,NaN,ENT,67.0
106591,Pindulo VDM (Pty) Ltd,Road Speed Overspeeding,Default,NaN,NaN,8.914219e+17,-9.174768e+18,2020-09-10T10:17:42,2.804790e+18,-28.244055,23.559973,70.0,0.0,-2864371101042705250,-7341754787231762831,0.0,NaN,NaN,ENT,70.0
106592,Pindulo VDM (Pty) Ltd,Road Speed Overspeeding,Default,NaN,NaN,8.914219e+17,-9.174768e+18,2020-09-10T10:18:43,2.804790e+18,-28.255138,23.559723,69.0,0.0,-2864371101042705250,-7341754787231762831,0.0,NaN,NaN,ENT,69.0
106593,Pindulo VDM (Pty) Ltd,Road Speed Overspeeding,Default,NaN,NaN,8.914219e+17,-9.174768e+18,2020-09-10T11:50:21,2.804814e+18,-28.537834,24.528334,57.0,0.0,-2864371101042705250,-7341754787231762831,231481.5,NaN,NaN,ENT,57.0


In [82]:
set(events_df[events_df.DriverId_u0==gkk.iloc[3].DriverId_u0].EventTypeId)

{-6962998577837367602,
 -3890646499157906515,
 -2864371101042705250,
 4750800303282680186}

In [122]:
df = pd.DataFrame([('bird', 'Falconiformes', 389.0),
('bird', 'Psittaciformes', 24.0),
('mammal', 'Carnivora', 80.2),
('mammal', 'Primates', np.nan),
('mammal', 'Carnivora', 58)],
index=['falcon', 'parrot', 'lion', 'monkey', 'leopard'],
columns=('class', 'order', 'max_speed'))

In [123]:
df

,class,order,max_speed
falcon,bird,Falconiformes,389.0
parrot,bird,Psittaciformes,24.0
lion,mammal,Carnivora,80.2
monkey,mammal,Primates,NaN
leopard,mammal,Carnivora,58.0


In [124]:
grouped = df.groupby('class')

In [125]:
grouped

In [126]:
grouped = df.groupby('order', axis='columns')
grouped = df.groupby(['class', 'order'])